In [ ]:
!pip install RDkit
!pip install -q tensorflow-gnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_gnn as tfgnn
import IPython.display as display

In [ ]:
data_train = pd.read_csv('/content/drive/MyDrive/forth_dataset/train_onlySMILES_shuffle.csv')
data_predict = pd.read_csv('/content/drive/MyDrive/fifth_dataset/fda_fixed_onlySMILES.csv')
sm_train=[]
lb=[]
sm_predict=[]
for i in range(len(data_train)):
    sm_train.append(data_train.iloc[i].values[0])
    lb.append(data_train.iloc[i].values[1])
for i in range(len(data_predict)):
    sm_predict.append(data_predict.iloc[i].values[1])


In [ ]:
for itr in range(10):
    num=0
    a=0
    b=0
    for i in range(283):
        if(i%10==itr):
            if(lb[i]==1):
                a+=1
            else:
                b+=1
            num+=1
    print(a,b,num)
    print()

In [ ]:
print(len(data_train))

In [ ]:
print(len(data_predict))

In [ ]:
def process(sm,n):

    ms=[Chem.MolFromSmiles(sm[n])]
    element = []
    elementcopy = []
    mtrx = [Chem.rdmolops.GetAdjacencyMatrix(m,useBO=True) for m in ms]

    for i in range(len(mtrx)):
        element = [atom.GetSymbol() for atom in ms[i].GetAtoms()]
        elementcopy=element.copy()

    #0=C, 1=O, 2=N, 3=P, 4=S, 5=F, 6=Cl

    element_res=np.zeros((len(element),7))
    idx=0

    for i in element:
        if(i=='C'):
            element_res[idx][0]=1
        if(i=='O'):
            element_res[idx][1]=1
        if(i=='N'):
            element_res[idx][2]=1
        if(i=='P'):
            element_res[idx][3]=1
        if(i=='S'):
            element_res[idx][4]=1
        if(i=='F'):
            element_res[idx][5]=1
        if(i=='Cl'):
            element_res[idx][6]=1
        idx=idx+1

   ###################################################################

    edge_source=[]
    edge_target=[]
    edge_num=np.zeros((0,4))

    for idx1 in range(len(element)):
        for idx2 in range(len(element)):
            if(mtrx[0][idx1][idx2] != 0):
                edge_source.append(idx1)
                edge_target.append(idx2)
                arr=[[0,0,0,0]]
                if(mtrx[0][idx1][idx2]==1.5):
                    arr[0][0]=1
                    arr[0][1]=1
                else:
                    arr[0][int(mtrx[0][idx1][idx2])-1]=1
                edge_num=np.append(edge_num, arr, axis=0)
    return element_res,edge_source,edge_target,edge_num

In [ ]:
###########################################################################

In [ ]:
def _2D_to_1D(vals):
    ret=[]
    for i in vals:
        for j in i:
            ret.append(j)
    return ret

In [ ]:
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value))

def _float_1D_feature(values):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value for value in values]))

def _float_2D_feature(values):
   ret=_2D_to_1D(values)
   return tf.train.Feature(float_list=tf.train.FloatList(value=[value for value in ret]))

def _int64_0D_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_1D_feature(values):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value for value in values]))

def _int64_2D_feature(values):
  ret=_2D_to_1D(values)
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value for value in ret]))


In [ ]:
@tf.py_function(Tout=tf.string)
def train_serialize_example(label,edge_sz,edge_so,edge_tg,edge_hs,node_sz,atom_hs):

  feature = {
      'context/label': _int64_0D_feature(label),
      'edges/bonds.#size': _int64_0D_feature(edge_sz),
      'edges/bonds.#source': _int64_1D_feature(edge_so),
      'edges/bonds.#target': _int64_1D_feature(edge_tg),
      'edges/bonds.hidden_state': _float_2D_feature(edge_hs),
      'nodes/atoms.#size': _int64_0D_feature(node_sz),
      'nodes/atoms.hidden_state': _float_2D_feature(atom_hs)
  }

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
@tf.py_function(Tout=tf.string)
def predict_serialize_example(edge_sz,edge_so,edge_tg,edge_hs,node_sz,atom_hs):

  feature = {
      'edges/bonds.#size': _int64_0D_feature(edge_sz),
      'edges/bonds.#source': _int64_1D_feature(edge_so),
      'edges/bonds.#target': _int64_1D_feature(edge_tg),
      'edges/bonds.hidden_state': _float_2D_feature(edge_hs),
      'nodes/atoms.#size': _int64_0D_feature(node_sz),
      'nodes/atoms.hidden_state': _float_2D_feature(atom_hs)
  }

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
for itr in range(1,11):

    filename_train = 'train_smiles_'+str(itr)+'.tfrecord'
    filename_val = 'val_smiles_'+str(itr)+'.tfrecord'

    with tf.io.TFRecordWriter(filename_train) as writer, tf.io.TFRecordWriter(filename_val) as writer2:

        for i in range(283):

            (element_res,edge_source,edge_target,edge_num)=process(sm_train,i)
            example = train_serialize_example(lb[i],len(edge_source),edge_source, edge_target, edge_num,len(element_res),element_res)

            if(i%10==(itr-1)):
                writer2.write(example.numpy())
            else:
                writer.write(example.numpy())


In [ ]:
filename_train_all = 'all_train_smiles.tfrecord'
with tf.io.TFRecordWriter(filename_train_all) as writer:
  for i in range(283):
    (element_res,edge_source,edge_target,edge_num)=process(sm_train,i)
    example = train_serialize_example(lb[i],len(edge_source),edge_source, edge_target, edge_num,len(element_res),element_res)
    writer.write(example.numpy())

In [ ]:
filename_predict = 'fda_fixed_1.tfrecord'
with tf.io.TFRecordWriter(filename_predict) as writer:
    for i in range(len(data_predict)):
        (element_res,edge_source,edge_target,edge_num)=process(sm_predict,i)

        #it should use predict_serialize_example()
        example = train_serialize_example(1,len(edge_source),edge_source, edge_target, edge_num, len(element_res),element_res)

        writer.write(example.numpy())